In [1]:
]link.create duck ../duckdb-dyalog
]link.create caruna ../caruna-dyalog
]link.create hc ../dyalog-httpcommand

Linked: #.duck ←→ /home/kimmo/dyalog-duckdb-parquet/../duckdb-dyalog

Linked: #.caruna ←→ /home/kimmo/dyalog-duckdb-parquet/../caruna-dyalog

Linked: #.hc ←→ /home/kimmo/dyalog-duckdb-parquet/../dyalog-httpcommand

In [2]:
year2022←caruna.Fetch 2022 2022

In [3]:
5↑¨year2022

┌──────────────────────────────────────────────────────────────────────────────
│┌───────────────────┬───────────────────┬────────────────┬────────────────┬───
││2021 12 31 22 0 0 0│2021 12 31 23 0 0 0│2022 1 1 0 0 0 0│2022 1 1 1 0 0 0│202
│└───────────────────┴───────────────────┴────────────────┴────────────────┴───
└──────────────────────────────────────────────────────────────────────────────
 ──────────────┬────────────────────────┐
 ─────────────┐│3.21 5.71 9.32 8.01 4.93│
 2 1 1 2 0 0 0││ │
 ─────────────┘│ │
 ──────────────┴────────────────────────┘

In [4]:
duck.db.init '/home/kimmo/duckdb-dyalog/lib/'

In [5]:
_db←duck.db.open ':memory:'
_con←duck.db.connect _db

In [6]:
_←duck.db.query _con 'CREATE TABLE consumption (time TIMESTAMP, consumption DOUBLE)'
duck.db.append _con '' 'consumption' year2022  duck.db.type.(TIMESTAMP DOUBLE)

Success!

In [7]:
_←duck.db.query _con 'CREATE TABLE prices (time TIMESTAMP, price DOUBLE)'
duck.db.append _con '' 'prices' (caruna.getElecPrice 2022)  duck.db.type.(TIMESTAMP DOUBLE)

Success!

In [8]:
]multiline
_←duck.db.query _con ```
CREATE TABLE electricity AS 
SELECT 
    c.time,
    c.consumption,
    p.price
FROM consumption c, prices p 
WHERE c.time=p.time
```

In [9]:
]multiline
duck.db.toTable duck.db.query _con ```
SELECT 
    year(time) as year,
    month(time) as month,
    sum(consumption) as consumption,
    sum(price*consumption/100) as costs,
    avg(price) as avg_price,
    sum(price*consumption)/sum(consumption) as weighted_price 
FROM electricity 
WHERE year(time)>2021 
GROUP BY year(time),month(time)
```

┌────┬─────┬───────────┬───────────┬───────────┬──────────────┐
│year│month│consumption│costs │avg_price │weighted_price│
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│1 │3906.38 │511.4491408│13.24715667│13.09266228 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│2 │3078.02 │306.7896531│10.0258244 │9.967110452 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│3 │2777.89 │286.8994659│10.74263833│10.32796352 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│4 │2145.95 │195.0728148│9.8475375 │9.090277721 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│5 │1754.37 │254.6228395│16.411175 │14.51363393 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│6 │1456.76 │216.1720991│17.40090278│14.83923907 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│7 │1523.36 │319.4839924│22.80603833│20.97232384 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│8 │1349.76 │409.8211578│32.43009333│30.36252058 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│9 │1274.14 │313.2125894│26.71756528│24.58227427 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│10 │1076.59 │143.7606438│13.94566866│13.35333264 │
└────┴─────┴───────────┴───────────┴───────────┴──────────────┘

In [10]:
duck.db.query _con 'COPY (SELECT * FROM electricity) TO ''electricity.parquet'' (FORMAT ''parquet'')'

┌───────┬────┬─┐
│┌─────┐│7054│5│
││Count││ │ │
│└─────┘│ │ │
└───────┴────┴─┘

In [11]:
_←duck.db.disconnect _con
_←duck.db.close _db 